In [27]:
%matplotlib notebook
import sqlite3
import cufflinks as cf
import pandas as pd
import matplotlib
from plotly import plotly

# Load data
db = sqlite3.connect("../confs.db")
speakers = pd.read_sql_query("SELECT * from speakers", db)

cf.set_config_file(theme='polar')

In [2]:
speakers['country'].value_counts().iplot(kind='bar', filename='speakers-country')

In [136]:
df = pd.read_sql_query("""
SELECT s.*, t.tag from sessions s 
LEFT JOIN session_tags t ON t.session = s.id
""", db)

mapping = {
    'js': 'javascript',
    'pm': 'project-management',
    'accessib': 'accessibility',
    'cloud': 'cloud-computing',
    'data': 'data-science',
    'front-end': 'frontend',
    'db': 'databases',
    'qa': 'testing-qa',
}
df['tag'] = df['tag'].map(lambda t: mapping.get(t) or t)
df['tag'].value_counts()[:15].iplot(kind='bar', filename='sessions-tags')

In [137]:
import re

df['year'] = df.apply(lambda r: int(re.search(r'\d+', r.edition).group(0)), axis=1)

pivot = df.pivot_table(index='year', columns='tag', values='id', fill_value=0, aggfunc='count')
pivot_sum = pivot.sum(axis=1)
pivot = pivot.apply(lambda c: c/pivot_sum)
pivot = pivot.applymap(lambda x: f'{x*100:0.2f}')
pivot.iplot(filename='tag-per-year')